In [ ]:
import matplotlib.pyplot as plt
import random
import itertools
import numpy as np

In [449]:
# algorithm parameters
numCoeffs = 41
populationSize = 100
generations = 50
mutationRate = 0.15
functionRange = (-np.pi, np.pi)
sampleCount = 100

In [450]:
# These functions are given as samples to use in the algorithm
def getTargetFunction(functionName="sin_cos"):
    def sinCosFunction(t):
        """Target function: sin(2πt) + 0.5*cos(4πt)."""
        return np.sin(2 * np.pi * t) + 0.5 * np.cos(4 * np.pi * t)

    def linearFunction(t):
        """Simple linear function: y = 2t + 1."""
        return 2 * t + 1

    def quadraticFunction(t):
        """Quadratic function: y = 4t^2 - 4t + 2."""
        return 4 * (t**2) - 4 * t + 2

    def cubicFunction(t):
        """Cubic function: y = 8t^3 - 12t^2 + 6t."""
        return 8 * (t**3) - 12 * (t**2) + 6 * t

    def gaussianFunction(t):
        """Gaussian function centered at t=0.5."""
        mu = 0.5
        sigma = 0.1  # Adjust sigma to control the width of the peak
        return np.exp(-((t - mu) ** 2) / (2 * sigma**2))

    def squareWaveFunction(t):
        """Approximation of a square wave. Smoothed for better Fourier approximation."""
        return 0.5 * (np.sign(np.sin(2 * np.pi * t)) + 1)

    def sawtoothFunction(t):
        """Sawtooth wave, normalized to [0, 1]."""
        return (t * 5) % 1

    def complexFourierFunction(t):
        return (
            np.sin(2 * np.pi * t)
            + 0.3 * np.cos(4 * np.pi * t)
            + 0.2 * np.sin(6 * np.pi * t)
            + 0.1 * np.cos(8 * np.pi * t)
        )

    def polynomialFunction(t):
        return 10 * (t**5) - 20 * (t**4) + 15 * (t**3) - 4 * (t**2) + t + 0.5

    functionOptions = {
        "sin_cos": sinCosFunction,
        "linear": linearFunction,
        "quadratic": quadraticFunction,
        "cubic": cubicFunction,
        "gaussian": gaussianFunction,
        "square_wave": squareWaveFunction,
        "sawtooth": sawtoothFunction,
        "complex_fourier": complexFourierFunction,
        "polynomial": polynomialFunction,
    }

    selectedFunction = functionOptions.get(functionName.lower())
    if selectedFunction:
        return selectedFunction

In [451]:
# generate samples
tSamples = np.linspace(functionRange[0], functionRange[1], sampleCount)
fSamples = getTargetFunction()(tSamples)

## 🧬 Section 1: Definition of Concepts

In this project, the goal is to use a genetic algorithm to approximate the Fourier series coefficients of a sampled unknown function \( f(t) \).

To achieve this, we need to define the genetic representation of potential solutions:

- **Gene:** A gene represents a single coefficient in the Fourier series.
  - There are two types of coefficients:
    - Cosine coefficients: \( a_0, a_1, ..., a_{20} \)
    - Sine coefficients: \( b_1, ..., b_{20} \)

- **Chromosome:** A chromosome is a full candidate solution, containing all 41 coefficients:
  \[
  [a_0, a_1, ..., a_{20}, b_1, ..., b_{20}]
  \]

- **Population:** A collection of chromosomes. The algorithm starts with a randomly generated population and evolves it using genetic operators.

- **Search space:** Since each gene is bounded by the interval \([-A, A]\), and we have 41 genes, the search space is a 41-dimensional bounded continuous space.

Each chromosome corresponds to a unique Fourier series approximation. The genetic algorithm searches this space to find the chromosome whose Fourier approximation best fits the sampled function \( f(t) \).


In [ ]:
# ---------- Parameters ----------
num_terms = 20
numCoeffs = 1 + num_terms * 2  # a0 + an + bn
gene_limit = 5  # Coefficients range [-5, 5]

# ---------- Chromosome Generator ----------
def create_random_chromosome():
    return [random.uniform(-gene_limit, gene_limit) for _ in range(numCoeffs)]


In this section, we defined the genetic encoding of the solution. A gene corresponds to one coefficient of the Fourier series, and a chromosome contains all 41 coefficients. The search space is continuous and high-dimensional. By sampling chromosomes randomly from the bounded interval 
[
−
𝐴
,
𝐴
]
[−A,A], we can initialize a diverse population.



## 🧬 Section 2: Creating the Initial Population

To begin the genetic algorithm process, we must first generate a diverse population of chromosomes. Each chromosome is a potential solution to the problem — a candidate set of Fourier series coefficients.

Each chromosome consists of:
- \( a_0 \): The DC component (constant term)
- \( a_1, a_2, ..., a_{20} \): Cosine coefficients
- \( b_1, b_2, ..., b_{20} \): Sine coefficients

Total: **41 genes (coefficients)**

We will initialize three different populations to compare their impact on algorithm convergence:
- Small population: 50 chromosomes
- Medium population: 100 chromosomes
- Large population: 200 chromosomes

Later, we will analyze how population size affects the fitness progression.


In [ ]:
# ---------- Chromosome and Population ----------
def create_random_chromosome():
    return [random.uniform(-gene_limit, gene_limit) for _ in range(numCoeffs)]

def initialize_population():
    return [create_random_chromosome() for _ in range(populationSize)]


Running GA for Population Size: 50


NameError: name 'fitness_regularized' is not defined

In this section, we created initial populations with varying sizes (50, 100, 200 chromosomes), each containing 41 randomly generated Fourier coefficients. This initial diversity plays a critical role in determining how well and how fast the algorithm explores the search space.

Larger populations typically offer better exploration and convergence potential, but they also require more computation. We will analyze and compare their effects later by plotting fitness over generations.

## 📏 Section 3: Compatibility Measurement Criteria

To evaluate how well a chromosome approximates the target function, we need to define a **fitness (compatibility) metric**.

In function approximation problems, several common error metrics are used to quantify the difference between the predicted output \( \hat{f}(t) \) and the true output \( f(t) \).

### 🔸 1. RMSE – Root Mean Square Error
\[
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (f(t_i) - \hat{f}(t_i))^2}
\]
- Sensitive to large errors
- Common in regression and signal processing
- Easy to interpret in the original scale of the data

### 🔸 2. MAE – Mean Absolute Error
\[
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |f(t_i) - \hat{f}(t_i)|
\]
- Measures average magnitude of errors
- Less sensitive to outliers than RMSE

### 🔸 3. \( R^2 \) – Coefficient of Determination
\[
R^2 = 1 - \frac{\sum (f(t_i) - \hat{f}(t_i))^2}{\sum (f(t_i) - \bar{f})^2}
\]
- Indicates the proportion of variance explained by the model
- Value ranges from \( -\infty \) to 1 (1 = perfect fit)

---

### ✅ Selected Metric: **RMSE**

We choose RMSE as our primary fitness metric because:
- It penalizes larger errors more than MAE
- It is widely used in signal approximation and regression
- It gives better gradient behavior for optimization

In our genetic algorithm, we will use **negative RMSE** as the fitness value (so that higher fitness = lower error).


In [458]:
# Error Metrics

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def MAE(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def R_squared(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)


In this section, we compared three standard error metrics: RMSE, MAE, and R².
While MAE is robust and R² is statistically informative, we chose RMSE as the primary compatibility metric in our genetic algorithm due to its sensitivity to larger errors and widespread use in function approximation tasks.

## 🔁 Section 4: Crossover, Mutation, and Next Generation Selection Strategy

To evolve the population toward better solutions, we apply two main genetic operators:

---

### 🔸 Crossover

Crossover combines the genetic material of two parent chromosomes to create offspring. This helps propagate good traits and explore new combinations in the search space.

We implement two types:
1. **Uniform Crossover** – each gene is randomly taken from one of the two parents
2. **Single-Point Crossover** – a random split point is chosen; genes are exchanged after that point

Crossover rate (e.g., 90%) determines how often crossover is applied vs. copying the parents directly.

---

### 🔸 Mutation

Mutation introduces small random changes to a chromosome’s genes to:
- Maintain genetic diversity
- Escape local optima

The mutation rate (e.g., 10%) defines the chance that each gene will mutate.

We use **scaled mutation** where the mutation strength decreases with each generation:
\[
\text{scale} = \max(0.1, 1 - \frac{\text{generation}}{\text{total\_generations}})
\]

---

### 🔸 Elitism

To preserve high-quality solutions, we pass a percentage (e.g., 10%) of the best chromosomes **unchanged** to the next generation.

This ensures that the algorithm never "forgets" its best results.

---

Together, crossover, mutation, and elitism balance **exploration** (searching new regions) and **exploitation** (refining the best-known areas).


In [468]:
# ----- Crossover Methods -----
def uniform_crossover(parent1, parent2):
    if random.random() < crossoverRate:
        return [
            random.choice([p1, p2]) for p1, p2 in zip(parent1, parent2)
        ], [
            random.choice([p1, p2]) for p1, p2 in zip(parent2, parent1)
        ]
    else:
        return parent1[:], parent2[:]

def single_point_crossover(parent1, parent2):
    if random.random() < crossoverRate:
        point = random.randint(1, num_coeffs - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    else:
        return parent1[:], parent2[:]

# ----- Mutation -----
def mutation(chromosome, current_generation):
    scale = max(0.1, 1.0 - (current_generation / generations))  # decaying mutation scale
    for i in range(len(chromosome)):
        if random.random() < mutationRate:
            chromosome[i] += random.gauss(0, 0.3 * scale)
            chromosome[i] = np.clip(chromosome[i], -A, A)
    return chromosome

# ----- Tournament Selection -----
def tournament_selection(population, fitness_values, k=3):
    selected = random.sample(list(zip(population, fitness_values)), k)
    return max(selected, key=lambda x: x[1])[0]

# ----- Next Generation -----
def select_next_generation(population, fitness_values, current_generation,
                           selection_method='tournament',
                           crossover_method='uniform'):
    
    # Elitism: keep top X% unchanged
    num_elites = int(elitismRate * len(population))
    elites = [population[i] for i in np.argsort(fitness_values)[-num_elites:]]
    new_population = elites.copy()
    
    # Fill the rest of the population
    while len(new_population) < populationSize:
        # Selection
        parent1 = tournament_selection(population, fitness_values)
        parent2 = tournament_selection(population, fitness_values)
        
        # Crossover
        if crossover_method == 'uniform':
            child1, child2 = uniform_crossover(parent1, parent2)
        else:
            child1, child2 = single_point_crossover(parent1, parent2)

        # Mutation
        new_population.append(mutation(child1, current_generation))
        if len(new_population) < populationSize:
            new_population.append(mutation(child2, current_generation))

    return new_population


In this section, we defined the core genetic operators for evolving the population:

1. Crossover combines parents to create diverse offspring. We implemented both uniform and single-point crossover.

2. Mutation introduces variability and helps avoid local minima. We used a scaled Gaussian mutation with a decaying intensity.

3. Elitism ensures the best solutions are preserved between generations.

These components balance the need to explore new regions of the solution space while exploiting known good solutions.

## 📈 Section 5: Offspring Analysis

To evaluate the performance of the genetic algorithm, we analyze the following:

---

### 🔹 1. Fourier Approximation vs Target Function

After running the algorithm, we select the **best chromosome** and use it to reconstruct the Fourier series.  
We then **plot it against the true objective function** to visualize how well the algorithm has learned the underlying signal.

---

### 🔹 2. Fitness Progression Over Generations

We track and plot the **best fitness value at each generation**.  
This reveals how quickly the algorithm converges and whether it continues to improve or stagnates.

---

### 🔹 3. Multiple Objective Functions

To test the algorithm’s generalization, we repeat this process for multiple objective functions such as:
- A basic sinusoidal combination (`sin_cos`)
- A Gaussian bell-shaped curve (`gaussian`)
- A polynomial (`polynomial`)

This helps us understand how well the algorithm approximates **periodic vs non-periodic functions**.


In [476]:

def fitness_regularized(chromosome, t, y_true, alpha=regularization_alpha):
    y_pred = evaluate_fourier_series(chromosome, t)
    error = np.mean((y_pred - y_true) ** 2)
    regularization = np.mean(np.square(chromosome))
    return -(error + alpha * regularization)

def run_and_plot(f_target, label="Function", pop_size=100, gens=100):
    # Initialize population
    population = [create_random_chromosome() for _ in range(pop_size)]
    fitness_history = []

    # True values
    t = np.linspace(0, 1, sampleCount)
    f_true = f_target(t)

    for gen in range(gens):
        fitness_values = [fitness_regularized(chromo, t, f_true) for chromo in population]
        best_idx = np.argmax(fitness_values)
        best_chromo = population[best_idx]
        fitness_history.append(fitness_values[best_idx])
        population = select_next_generation(population, fitness_values, gen)

    # Final prediction
    f_approx = evaluate_fourier_series(best_chromo, t)

    # Plot results
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(t, f_true, label="Target Function")
    plt.plot(t, f_approx, linestyle="--", label="Fourier Approximation")
    plt.title(f"{label} - Function vs Approximation")
    plt.xlabel("t")
    plt.ylabel("f(t)")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(fitness_history)
    plt.title(f"{label} - Fitness Progression")
    plt.xlabel("Generation")
    plt.ylabel("Best Fitness")
    plt.grid(True)

    plt.tight_layout()
    plt.show()


NameError: name 'regularization_alpha' is not defined

In [477]:
def run_and_plot(f_target, label="Function", pop_size=100, gens=100):
    population = [create_random_chromosome() for _ in range(pop_size)]
    fitness_history = []

    t = np.linspace(0, 1, sampleCount)
    f_true = f_target(t)

    for gen in range(gens):
        fitness_values = [fitness_regularized(chromo, t, f_true) for chromo in population]
        best_idx = np.argmax(fitness_values)
        best_chromo = population[best_idx]
        fitness_history.append(fitness_values[best_idx])
        population = select_next_generation(population, fitness_values, gen,
                                            selection_method='tournament',
                                            crossover_method='uniform')

    f_approx = evaluate_fourier_series(best_chromo, t)

    # Plot
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(t, f_true, label="Target Function")
    plt.plot(t, f_approx, linestyle="--", label="Fourier Approximation")
    plt.title(f"{label} - Function vs Approximation")
    plt.xlabel("t")
    plt.ylabel("f(t)")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(fitness_history)
    plt.title(f"{label} - Fitness Progression")
    plt.xlabel("Generation")
    plt.ylabel("Best Fitness")
    plt.grid(True)

    plt.tight_layout()
    plt.show()


In [481]:

f = getTargetFunction("sin_cos")
run_and_plot(f, label="sin".title())

NameError: name 'fitness_regularized' is not defined